# MNIST model Tester

## Design Criteira

Before we jump in to design discussions. Here are some cavets!

- I'm not a UI developer or a Python Nija!
- I know using Wt (C++ lib) to build the Web application is not the best way to go for such simple applications.

With this excercise I'm trying to demonstrate my skill's which I'm good at and improve my skills on the other programming languages and try to build the thing better each time. I would love to build a web app using python flask or nodejs or Anjularjs. But those are not my strenths, my strenth lies in C++ and hence I opted for Wt as choice of web app development for this excercise. I had little experience developing Web app using Wt. I developed/developing a tool called [Spookfish](https://github.com/Santhosh-KS/spookfish). So instead of spending time in learning new technology in 4 days of time and build half cooked app, I rather prefered to build a good quality app with my current strenths. 

With these caveats in mind lets dive in to the design aspects.

Here is the list of things which will be there in my mind when I try to build anything.

If you have not seen the [quick demo](https://www.youtube.com/watch?v=rRr-dCAFBMk&feature=youtu.be). Please watch it now before you proceed.

- Simplicity and Ease of Use.(For UI)
- Modular and maintainable code.
- Scalability.

Let me explain them in detail here.

### Simplicity

I'm a big fan of Apple products. I love their design. They design things so simple but yet so powerful. Apple design is very user friendly. Yet I don't own an Apple device! :) 

Jokes apart. When it comes to user experiance with UI, over the years, I've learnt to keep it simple. For example when you load the web app, there is nothing more you can do. It just has two widgets. One input field and second is just a button. This helps the user to focus on what he/she needs to do. Just put some valid error handling on those fields. This simple approach will remove lot of clutter in your code. Just enabling the complete webpage and asking user to fill in the details will always open a can of worms in the code. So I tried to keep the UI simple and effective for the user.


### Modular and maintainable code.

When I thought of building this web app, I wanted it to be reuseable. The models which we are testing in this example might be MNIST models. But this could be any generic model. If we hardcode the model names as part of the code then again we have to re-visit the code and compile and test etc. Which is very time consuming effort. To solve these kind of problem I've kept the design in to three parts.

- Client and server code 
- UI dependency code.
- Serve side of the code

I've kept the client and server code separate as these could be used and tested independently of any other components. You can find the example standalone code in **[example](https://github.com/Santhosh-KS/mnistModelTester/tree/master/example)** directory.

Wanted to keep the UI dependency as much minimal as possible. Hence kept the URL parser and Model combo box selector separate.

Because I've kept the URL parser separete the URL's can be tested independently. This will help us to do do exclusive testing on the input URL from the user.**[unittests](https://github.com/Santhosh-KS/mnistModelTester/tree/master/unitTests)** 

This also helps to stick with our simplicity principle which we discussed earlier.

One more example for the modular approach is to keeping the dropdown options for the models and their names independent of UI. Just by modifying the [Models.json](https://github.com/Santhosh-KS/mnistModelTester/blob/master/deploy/data/Models.json) file we will be able to give any name to any of the models. Or introduce new models in to the UI without touching the code. All we need to do is just edit the Models.json file rest of the things are taken care automatically on the client side of the code. Ofcourse you need to add the support for the new model on the server side. Which you have to do in any case. Keeping the models in Models.json file makes the code lightweight and much more easily maintainable. Say if we decide to change the name of the model, just change it in json file.


Server side of the code just parses the json request sent by client and calls an internal python modules. This is bascially my limitation which I've to workthrough. Currently I'm not completely aware of how to build the Models using C++ and most of the ML models are built using Python. And Python has a wide set of libraries available, which will take a lifetime for me alone to build all of it by myself. Hence I've made a compromise of calling a python code as script called [modelLoader.py](https://github.com/Santhosh-KS/mnistModelTester/blob/master/deploy/scripts/modelLoader.py). This python module does all the heavy lifting. 

As we can see I try to incorpore my design principles as much as possible by keeping the code modular.


### Scalability

There are currently few limitations with the current design to scale it to deployable state where it can handle millions of requests per minute.

- Performace of the [modelLoader.py](https://github.com/Santhosh-KS/mnistModelTester/blob/master/deploy/scripts/modelLoader.py)

In this script I'm storing the each image URL's in to the disk and then reading it again. Ideally we shouldn't be doing lot of I/O operations. This will have very bad user experiance. If a user gives a very huge image of >50MB then we are going to waste few seconds writing it to the disk and few more seconds reading it back. This will ultimately show up as, user has to wait for a longer time on the page to get his results. 
I would have written a code which would avoid the reading and writing of images to the disk. But that comes with the cost of time and complexity of the code. It would take longer time to handle different image formats and need to put in lot of effort in keeping it modular. This is the compromise which I've made in my current design.

- Dynamically loaded ML models.

In our current design we are currently loading and uloading the the stored models aka .pkl files or .h5 files. This again incurs more I/O on the system which results in delayed response.
This again can be avoided by loading all the models only once and just predict the given image. Which means more C++/Python modules running in parallel to handle such situations.

These are the two major bottlenecks which will hinder our scalability aspect of our design. But I have tried to give a fair explaination on why I've done what I've done in the current code.

Apart from that, as with any Scalable module we need to have the Loadbalancer to handle the traffic.
Also an caching mechanism to remember the previously shared image URL will help in retriving the results faster and adds bonus points to our scalability aspect.

### Monitoring the performance of our model.

One easiest thing to do is to run the test image on perfect ML model with the Model we are testing.

Ex:

|#|URL|Perfect Model Prediction| Test Model predictio|
|--|---|------------------------|-----|
|1|http://1.png| 1 | 1|
|2|http://6.png| 6 | 5|
|1|http://7.png| 7 | 9|
|1|http://2.png| 2 | 2|

Here we are showing it as table. Same table can be stored in a Database. We can compare the results which we see over a period of time to monitor the performance of our model.

The above method assumes we already have some perfect model. For example MNIST data model. There are n-number of models which can give 100% accuracy. We can compare our model against those models. 
But what if we don't have those perfect models? I'm not sure. May be we can try some image agumentation.

For example here is the list of sample **image agumentation** we can do.
- rotate the image say about $5\,^{\circ}$ to the left or $5\,^{\circ}$ to the right
- Add a slight blurring effect on the image.
- Add some random noice on the image.
- Slightly crop the image. without losing the digits in the center.
- Dilate/errode the image
etc. 

There is no limit on the options available on image augumentation.

**NOTE**: We need to be extreamly careful about what type of agumentation we are doing on MNIST kind of data set. For example, we should not do flip or rotate on the MNIST data set or the dataset where we are trying to classify the text. If we rotate an image of **6 by $180\,^{\circ}$ then image becomes 9**. Which is not correct. Similarly if we flip digit **7** it loses its meaning. So we need to be careful in identifying the type of agumentation which are willing to apply on the images.

Apply the above mentioned agumentation methods on the image and feed it to the ML model for a period of time and randomly sample those results and pass the same images to check the current prediction and valdiate it with its previous values?

Not completely sure how that will really help. But my gut feeling says it could work :)

But Image agumentation will cetainly help if have very small dataset as with our 500 samples only MNIST models. I wish I had little more time to complete the task, I would have definately tried it.

Currently the here where my model's accuracy stands.

|#|Model|Accuracy|
|-|-----|--------|
|1|Logistic Regression| 81%|
|2|KNN| 80%|
|3|Logistic Regression| 60-77%|
